# License Plate Recognition using TAO LPRNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained baseline18 LPRNet model and train it on the OpenALPR benchmark dataset
* Run Inference on the trained model
* Export the trained model to a .etlt file for deployment to DeepStream
* Run inference on the exported. etlt model to verify deployment using TensorRT

## Table of Contents

This notebook shows an example usecase of LPRNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
7. [Deploy](#head-7)
8. [Verify the deployed model](#head-8)

## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/lprnet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*


In [1]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variable with your key.")
%env KEY=nvidia_tlt
%env GPU_INDEX=0
!printf "\nInternal Paths for Tao Workspace:\n\n" 
%env INT_MAIN_DIR=/workspace/tao-experiments/lprnet
%env INT_DATA_DIR=/workspace/tao-experiments/lprnet/data
%env INT_SPECS_DIR=/workspace/tao-experiments/lprnet/specs

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/detectnet_v2

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/lprnet
os.environ["LOCAL_PROJECT_DIR"] = os.getcwd()

os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "experiments"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "data"
)

# Showing list of specification files.
# !ls -rlt $LOCAL_SPECS_DIR
!printf "\nPaths for local machine:\n\n" 
!echo LOCAL_PROJECT_DIR:    $LOCAL_PROJECT_DIR
!echo LOCAL_DATA_DIR:       $LOCAL_DATA_DIR
!echo LOCAL_EXPERIMENT_DIR: $LOCAL_EXPERIMENT_DIR
!echo LOCAL_SPECS_DIR:      $LOCAL_SPECS_DIR

Please replace the variable with your key.
env: KEY=nvidia_tlt
env: GPU_INDEX=0

Internal Paths for Tao Workspace:

env: INT_MAIN_DIR=/workspace/tao-experiments/lprnet
env: INT_DATA_DIR=/workspace/tao-experiments/lprnet/data
env: INT_SPECS_DIR=/workspace/tao-experiments/lprnet/specs

Paths for local machine:

LOCAL_PROJECT_DIR: /home/pepeleduin/mobia/mobia-project/training/lprnet
LOCAL_DATA_DIR: /home/pepeleduin/mobia/mobia-project/training/lprnet/data
LOCAL_EXPERIMENT_DIR: /home/pepeleduin/mobia/mobia-project/training/lprnet/experiments
LOCAL_SPECS_DIR: /home/pepeleduin/mobia/mobia-project/training/lprnet/specs


The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [2]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": os.environ["INT_MAIN_DIR"]
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["INT_SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [3]:
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "/home/pepeleduin/mobia/mobia-project/training/lprnet",
            "destination": "/workspace/tao-experiments/lprnet"
        },
        {
            "source": "/home/pepeleduin/mobia/mobia-project/training/lprnet/specs",
            "destination": "/workspace/tao-experiments/lprnet/specs"
        }
    ]
}

In [5]:
!pip3 install --upgrade pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 2.1 MB 679 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 11.2 MB/s eta 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 58.0.4
    Uninstalling setuptools-58.0.4:
      Successfully uninstalled setuptools-58.0.4
  Attempting uninstall: pip
    Found existing installation: pip 21.2.2
    Uninstalling pip-21.2.2:
      Successfully uninstalled pip-21.2.2


In [6]:
# Installing the dependencies
import os
os.environ["PROJECT_DIR"] = os.getcwd()
!pip3 install -r $PROJECT_DIR/../requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in the `nvidia-pyindex` python index. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.6.9 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python >=3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the virtualenv and virtualenvwrapper packages.

In [7]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-pyindex
!pip3 install nvidia-tao

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8415 sha256=dd065ca3a3021992b42e3ae4ddd9c3229d35dd2e6e9be99072f6bfe15bbf1a29
  Stored in directory: /tmp/pip-ephem-wheel-cache-ihpjcagm/wheels/f1/a1/a1/6cc45cc1ae6b1876f12ef399c0d0d6e18809e9ced611c7c2a7
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 KB 881.8 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 10.0 MB

In [8]:
# View the versions of the TAO launcher
!tao info

Configuration of the TAO Toolkit Instance
dockers: ['nvidia/tao/tao-toolkit-tf', 'nvidia/tao/tao-toolkit-pyt', 'nvidia/tao/tao-toolkit-lm']
format_version: 2.0
toolkit_version: 3.22.02
published_date: 02/28/2022


## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

 We will be using the OpenALPR benchmark dataset for the tutorial. The following script will download the dataset automatically and convert it to the format used by TAO. 

In [9]:
!echo $LOCAL_DATA_DIR

/home/pepeleduin/mobia/mobia-project/training/lprnet/data


In [10]:
# Create local dir
!mkdir -p $LOCAL_DATA_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR

In [11]:
!bash download_and_prepare_data.sh $LOCAL_DATA_DIR

+ '[' -z /home/pepeleduin/mobia/mobia-project/training/lprnet/data ']'
++ pwd
+ CURRENT_DIR=/home/pepeleduin/mobia/mobia-project/training/lprnet
+ echo 'Cloning OpenALPR benchmark directory'
Cloning OpenALPR benchmark directory
+ '[' '!' -e benchmarks ']'
+ OUTPUT_DIR=/home/pepeleduin/mobia/mobia-project/training/lprnet/data
+ mkdir -p /home/pepeleduin/mobia/mobia-project/training/lprnet/data
+++ readlink -f download_and_prepare_data.sh
++ dirname /home/pepeleduin/mobia/mobia-project/training/lprnet/download_and_prepare_data.sh
+ SCRIPT_DIR=/home/pepeleduin/mobia/mobia-project/training/lprnet
+ echo 'Preprocessing OpenALPR benchmarks data for US'
Preprocessing OpenALPR benchmarks data for US
+ python3 /home/pepeleduin/mobia/mobia-project/training/lprnet/preprocess_openalpr_benchmark.py --input_dir=/home/pepeleduin/mobia/mobia-project/training/lprnet/benchmarks/endtoend/us/ --output_dir=/home/pepeleduin/mobia/mobia-project/training/lprnet/data
Total 222 samples in benchmark dataset
111 

In [12]:
# verify
!ls -l $LOCAL_DATA_DIR/
!ls -l $LOCAL_DATA_DIR/train
!ls -l $LOCAL_DATA_DIR/train/image
!ls -l $LOCAL_DATA_DIR/train/label

total 8
drwxr-xr-x 4 pepeleduin pepeleduin 4096 Apr  8 10:27 train
drwxr-xr-x 4 pepeleduin pepeleduin 4096 Apr  8 10:27 val
total 8
drwxr-xr-x 2 pepeleduin pepeleduin 4096 Apr  8 10:27 image
drwxr-xr-x 2 pepeleduin pepeleduin 4096 Apr  8 10:27 label
total 680
-rw-r--r-- 1 pepeleduin pepeleduin  1882 Apr  8 12:23 1e241dc8-8f18-4955-8988-03a0ab49f813.jpg
-rw-r--r-- 1 pepeleduin pepeleduin  1925 Apr  8 12:23 21d8c31d-3deb-494b-9c63-c0223306fd82.jpg
-rw-r--r-- 1 pepeleduin pepeleduin 10710 Apr  8 12:23 316b64c0-55bf-4079-a1c0-d93f461a576f.jpg
-rw-r--r-- 1 pepeleduin pepeleduin  2003 Apr  8 12:23 33fa5185-0286-4e8f-b775-46162eba39d4.jpg
-rw-r--r-- 1 pepeleduin pepeleduin  2092 Apr  8 12:23 37170dd1-2802-4e38-b982-c5d07c64ff67.jpg
-rw-r--r-- 1 pepeleduin pepeleduin  2067 Apr  8 12:23 4be2025c-09f7-4bb0-b1bd-8e8633e6dec1.jpg
-rw-r--r-- 1 pepeleduin pepeleduin  1860 Apr  8 12:23 5b562a61-34ad-4f00-9164-d34abb7a38e4.jpg
-rw-r--r-- 1 pepeleduin pepeleduin  2096 Apr  8 12:23 c9368c55-210d-456c-a5

## 2.1 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [15]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""),
                                         os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2022-04-08 12:25:08--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 18.64.174.116, 18.64.174.58, 18.64.174.47, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|18.64.174.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32588132 (31M) [application/zip]
Saving to: ‘/home/pepeleduin/mobia/mobia-project/training/lprnet/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  31.08M  4.17MB/s    in 7.7s    

2022-04-08 12:25:18 (4.02 MB/s) - ‘/home/pepeleduin/mobia/mobia-project/training/lprnet/ngccli/ngccli_cat_linux.zip’ saved [32588132/32588132]

Archive:  /home/pepeleduin/mobia/mobia-project/training/lprnet/ngccli/ngccli_cat_linux.zip
  inflating: /home/pepeleduin/mobia/mobia-project/training/lprnet/ngccli/ngc  
 extracting: /home/pepeleduin/mobia/mobia-project/training/lprnet/ngccli/ngc.md5  


In [16]:
!ngc registry model list nvidia/tao/lprnet:*

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| train | 99.67 | 120   | 1     | V100  | 221.1 | 221.0 | UPLOA | Aug   |
| able_ |       |       |       |       |       | 6 MB  | D_COM | 24,   |
| v1.0  |       |       |       |       |       |       | PLETE | 2021  |
| deplo | 99.67 | 120   | 1     | V100  | 110.1 | 110.0 | UPLOA | Aug   |
| yable |       |       |       |       |       | 9 MB  | D_COM | 24,   |
| _v1.0 |       |       |       |       |       |       | PLETE | 2021  |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+


In [17]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_lprnet_baseline18/

In [18]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/lprnet:trainable_v1.0 \
     --dest $LOCAL_EXPERIMENT_DIR/pretrained_lprnet_baseline18

Downloaded 221.13 MB in 1m 13s, Download speed: 3.03 MB/s               
----------------------------------------------------
Transfer id: lprnet_vtrainable_v1.0 Download status: Completed.
Downloaded local path: /home/pepeleduin/mobia/mobia-project/training/lprnet/experiments/pretrained_lprnet_baseline18/lprnet_vtrainable_v1-1.0
Total files downloaded: 4 
Total downloaded size: 221.13 MB
Started at: 2022-04-08 12:26:39.945037
Completed at: 2022-04-08 12:27:53.020607
Duration taken: 1m 13s
----------------------------------------------------


In [19]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0

Check that model is downloaded into dir.
total 226376
-rw------- 1 pepeleduin pepeleduin       200 Apr  8 11:14 ch_lp_characters.txt
-rw------- 1 pepeleduin pepeleduin 115963904 Apr  8 11:14 ch_lprnet_baseline18_trainable.tlt
-rw------- 1 pepeleduin pepeleduin        70 Apr  8 11:14 us_lp_characters.txt
-rw------- 1 pepeleduin pepeleduin 115832832 Apr  8 11:14 us_lprnet_baseline18_trainable.tlt


## 2. Provide training specification <a class="anchor" id="head-2"></a>

* Note the spec $SPEC_DIR/default_sepc.txt is for training on US license plates:
    * the max license plate length is 8;
        * You can change `max_label_length` in `lpr_config` to satisfy your own dataset.
    * the characters of US license plates are: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, G, H, I, J, K, L, M, N, P, Q, R, S, T, U, V, W, X, Y, Z 
        * You can change `characters_list_file` in `dataset_config` to set your own characters.
        * `characters_list_file` should contain all the characters in dataset. And one character takes one line. 

In [ ]:
!cat $LOCAL_SPECS_DIR/tutorial_spec.txt

In [ ]:
!cat $LOCAL_SPECS_DIR/us_lp_characters.txt

## 3. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [36]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned

In [20]:
print("For multi-GPU, change --gpus based on your machine.")
!tao lprnet train --gpus=1 --gpu_index=$GPU_INDEX \
                  -e $INT_SPECS_DIR/tutorial_spec.txt \
                  -r $INT_MAIN_DIR/experiments/experiment_dir_unpruned \
                  -k $KEY \
                  -m $INT_MAIN_DIR/experiments/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0/us_lprnet_baseline18_trainable.tlt

For multi-GPU, change --gpus based on your machine.
2022-04-08 12:28:06,357 [INFO] root: Registry: ['nvcr.io']
2022-04-08 12:28:06,421 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.5-py3
2022-04-08 12:28:06,462 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/pepeleduin/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.

2022-04-08 17:28:11,992 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/lprnet/scripts/train.py:57: The name tf.ConfigProto is deprecated. Please 

In [ ]:
print("""
To resume training from a checkpoint, set the -m option to be the .tlt you want
to resume from and --initial_epochs to be the epoch index of the resumed
checkpoint
""")
# !tao lprnet train --gpu_index=$GPU_INDEX \
#                   -e $SPECS_DIR/tutorial_spec.txt \
#                   -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
#                   -k $KEY \
#                   -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-01.tlt
#                   --initial_epoch 2

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights/

## 4. Evaluate trained models <a class="anchor" id="head-4"></a>

In [ ]:
!tao lprnet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                     -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-24.tlt \
                     -k $KEY

## 5. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the lprnet inference tool to generate inferences on the trained models and print the results. 

In [ ]:
# Running inference for detection on n images
!tao lprnet inference --gpu_index=$GPU_INDEX -i $DATA_DOWNLOAD_DIR/val/image \
                      -e $SPECS_DIR/tutorial_spec.txt \
                      -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-24.tlt \
                      -k $KEY 

## 6. Deploy! <a class="anchor" id="head-6"></a>

In [ ]:
# Export in FP32 mode. 
!mkdir -p $LOCAL_EXPERIMENT_DIR/export 
!tao lprnet export --gpu_index=$GPU_INDEX
                   -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-24.tlt \
                   -k $KEY \
                   -e $SPECS_DIR/tutorial_spec.txt \
                   -o $USER_EXPERIMENT_DIR/export/lprnet_epoch-24.etlt \
                   --data_type fp32 \
                   --engine_file $USER_EXPERIMENT_DIR/export/lprnet_epoch-24.engine

In [ ]:
# Verify the tensorrt engine accuracy on the validation dataset
!tao lprnet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                     -m $USER_EXPERIMENT_DIR/export/lprnet_epoch-24.engine \
                     --trt

In [ ]:
# Check if etlt model is correctly saved.
!ls -l $LOCAL_EXPERIMENT_DIR/export

Verify engine generation using the `tao-converter` utility included with the docker.

The `tao-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tao-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The tao-converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

If you choose to integrate your model into deepstream directly, please refer to [deepstream dev guide](https://docs.nvidia.com/metropolis/deepstream/dev-guide/index.html) for more details.

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $LOCAL_EXPERIMENT_DIR/export

In [ ]:
# Convert to TensorRT engine(FP16).
# Please use -p option to set the min_shape / opt_shape / max_shape for dynmaic batch engine.
!tao converter $USER_EXPERIMENT_DIR/export/lprnet_epoch-24.etlt \
                   -k $KEY \
                   -p image_input,1x3x48x96,4x3x48x96,16x3x48x96 \
                   -t fp16 \
                   -e $USER_EXPERIMENT_DIR/export/lprnet_epoch-24_dynamic_batch.engine

In [ ]:
print('Exported engine:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export/

## 7. Verify the deployed model <a class="anchor" id="head-7"></a>

Verify the converted engine.

In [ ]:
# Running evaluation on validation dataset
!tao lprnet evaluate  --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                      -m $USER_EXPERIMENT_DIR/export/lprnet_epoch-24_dynamic_batch.engine \
                      --trt        